## Mxnet MNIST Single Machine SageMaker Training Example

MNIST is a widely used dataset for handwritten digit classification. It consists of 70,000 labeled 28x28 pixel grayscale images of hand-written digits. The dataset is split into 60,000 training images and 10,000 test images. There are 10 classes (one for each of the 10 digits). The task at hand is to train a model using the 60,000 training images and subsequently test its classification accuracy on the 10,000 test images.



In [ ]:
from sagemaker import get_execution_role

#Bucket location to save your custom code in tar.gz format.
custom_code_upload_location = 's3://<bucket-name>/customcode/mxnet_mnist'
#Bucket location where results of model training are saved.
model_artifacts_location = 's3://<bucket-name>/artifacts'

role = get_execution_role()

The ```MXNet``` class allows us to run single machine, multi-machine, and GPU mxnet training on SageMaker. Below we create an MXNet object to run our mnist training, passing in an IAMRole name to allow SageMaker to access our AWS resources. We run SageMaker mxnet training on a single ```m4.xlarge```.

Please see the ```mnist.py``` script to learn more about how training is performed. The script is an adaptation of the mxnet MNIST tutorial, found here: https://mxnet.incubator.apache.org/tutorials/python/mnist.html

In [ ]:
from sagemaker.mxnet import MXNet

mnist_estimator = MXNet(entry_point='/home/ec2-user/sample-notebooks/sagemaker-python-sdk/mxnet_mnist/mnist.py',
                        role=role,
                        output_path=model_artifacts_location,
                        code_location=custom_code_upload_location,
                        train_instance_count=1, 
                        train_instance_type='ml.m4.xlarge',
                        hyperparameters={'learning_rate': 0.11})

- TODO: Make the ECR images this is using public

After we've constructed our MXNet object, we can fit it using data stored in S3. Below we run SageMaker training on two input channels: train and test.

During training, SageMaker makes this data stored in S3 available in the local filesystem where the mnist script is running. The ```mnist.py``` script simply loads the train and test data from disk.


In [ ]:
%%time
import boto3

region = boto3.Session().region_name
train_data_location = 's3://sagemaker-sample-data-{}/mxnet/mnist/train'.format(region)
test_data_location = 's3://sagemaker-sample-data-{}/mxnet/mnist/test'.format(region)

mnist_estimator.fit({'train': train_data_location, 'test': test_data_location})

After training, we use the MXNet object to build and deploy an MXNetPredictor object. This creates an sagemaker-hosted prediction service that we can use to perform inference. 

This allows us to perform inference on json encoded multi-dimensional arrays. 

In [ ]:
%%time

predictor = mnist_estimator.deploy(initial_instance_count=1,
                                   instance_type='ml.c4.xlarge')

We can now use this predictor to classify hand-written digits. Drawing into the image box loads the pixel data into a 'data' variable in this notebook, which we can then pass to the mxnet predictor. 

In [ ]:
from IPython.display import HTML
HTML(open("/home/ec2-user/sample-notebooks/sagemaker-python-sdk/mxnet_mnist/input.html").read())

In [ ]:
response = predictor.predict(data)
print('Raw prediction result:')
print(response)

labeled_predictions = list(zip(range(10), response[0]))
print('Labeled predictions: ')
print(labeled_predictions)

labeled_predictions.sort(key=lambda label_and_prob: 1.0 - label_and_prob[1])
print('Most likely answer: {}'.format(labeled_predictions[0]))

# (Optional) Delete the Endpoint

In [ ]:
print(predictor.endpoint)

In [ ]:
import sagemaker

sagemaker.Session().delete_endpoint(predictor.endpoint)